In [1]:
import json, base64
s = {
    "fileNames": ["train.csv", "test.csv"],
    "target": "Hourly_Production",
    "userId": "servtech.ai_julie",
    "groupId": "servtech.ai",
    "projectId": "p00226",
    "job_id": "20230814082335099"
}
s = json.dumps(s)

b = s.encode("UTF-8")
e = base64.b64encode(b)
argv = e.decode("UTF-8")

print(f'python run.py "{argv}"')

python run.py "eyJmaWxlTmFtZXMiOiBbInRyYWluLmNzdiIsICJ0ZXN0LmNzdiJdLCAidGFyZ2V0IjogIkhvdXJseV9Qcm9kdWN0aW9uIiwgInVzZXJJZCI6ICJzZXJ2dGVjaC5haV9qdWxpZSIsICJncm91cElkIjogInNlcnZ0ZWNoLmFpIiwgInByb2plY3RJZCI6ICJwMDAyMjYiLCAiam9iX2lkIjogIjIwMjMwODE0MDgyMzM1MDk5In0="


In [2]:
import pandas as pd
from tools import *
from plot import *
from descript import *
from traceback import format_exc
from log_config import Log
from datetime import datetime
import os, json, sys, base64, shutil, redis

root    = r"C:\Users\tzuli\Documents\python\1_AI\ChartPresent"

log = Log()
log_path = os.path.join(root, "logs")
os.makedirs(log_path, exist_ok = True)
date = datetime.now().strftime("%Y%m%d")
logging = log.set_log(filepath = os.path.join(log_path, f"{date}.log"), level = 2, freq = "D", interval = 1, backup = 0, name = "log")
logging.info("-" * 200)

In [3]:
# get parameters
input_ = argv
input_ = base64.b64decode(input_).decode("utf-8")
input_ = json.loads(input_)
logging.info(f"input = {input_}")


# get config data
with open(os.path.join(root, "config.json")) as f:
    config = json.load(f)


# set redis to pass progress
r = redis.Redis(host = config["REDIS_CONFIG"]["host"], port = config["REDIS_CONFIG"]["port"])


# get files path and parsers path
file_names, file_paths, parser_paths = get_path(input_, config)


# create folders to save charts
chart_path = create_folder(input_, config)
logging.info(f"save charts to {chart_path}")


# get datas and features
dfs, numericals, categories = get_df_feat(file_paths, parser_paths)


if (len(file_names) == 2):
    assert ((numericals[0] == numericals[1]) and (categories[0] == categories[1])), "The features of the two datasets are different."


# TODO: delete
feats = ["1_Turbine_Negative_Pressure.1", "2_Turbine_Negative_Pressure.1", "3_Vacuum_Pump_Motor_Side_Vibration", "4_Turbine_Negative_Pressure.1"]
for feat in feats:
    numericals[0].remove(feat)
    numericals[1].remove(feat)
    categories[0].append(feat)
    categories[1].append(feat)
dfs[0][feats] = dfs[0][feats].astype(object)
dfs[1][feats] = dfs[1][feats].astype(object)

In [4]:
# create charts
logging.info("create charts...")
target = input_["target"]
top = 30
progress = 0
progress_gap = (1 / 13) if (len(file_names) == 2) else (1 / 8)
for file_name, df, numerical, category in zip(file_names, dfs, numericals, categories):
    heatmap(file_name, df, numerical, top, chart_path, target)
    progress += progress_gap
    r.set("ChartPresent_percent", round(progress, 2))
    break